In [79]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [186]:
class_metrics = ["Path", "Name", "LCOM5", "NL", "NLE", "WMC", "CBO", "CBOI", "NII", "NOI",
                 "RFC", "CD", "CLOC", "DIT", "NOA", "NOC", "NOD", "NOP", "LOC",
                 "LLOC", "NA", "NM", "NLA", "NLM", "NOS", "TLOC", "TLLOC",
                 "TNA", "TNM", "TNLA", "TNLM", "TNOS"]

method_metrics = ["HCPL","HDIF","HEFF","HNDB","HPL","HPV","HTRP","HVOL","MI","McCC","NL","NLE","NII","NOI","CD","CLOC","DLOC","TCD","TCLOC","LLOC","LOC","NOS","NUMPAR","TLLOC","TLOC","TNOS"]

In [171]:
class_stats = pd.read_csv("../data/class_metrics.csv").describe()
method_stats = pd.read_csv("../data/method_metrics_all.csv").describe()

In [60]:
class_stats.columns[class_stats.loc['max'] != 0].sort_values()

Index(['CBO', 'CLOC', 'DIT', 'LCOM5', 'LLOC', 'LOC', 'NLM', 'NM', 'NOA', 'NOC',
       'NOD', 'NOI', 'NOP', 'NOS', 'RFC', 'TLLOC', 'TLOC', 'TNLM', 'TNM',
       'TNOS', 'WMC'],
      dtype='object')

In [182]:
method_stats.columns[method_stats.loc['max'] != 0].sort_values()

Index(['CLOC', 'DLOC', 'HCPL', 'HDIF', 'HEFF', 'HNDB', 'HPL', 'HPV', 'HTRP',
       'HVOL', 'LLOC', 'LOC', 'MI', 'McCC', 'NOI', 'NOS', 'NUMPAR', 'TCLOC',
       'TLLOC', 'TLOC', 'TNOS'],
      dtype='object')

In [217]:
with open('class_metrics_non_zero.txt', 'w') as f:
    f.write('\n'.join(list(class_stats.columns[class_stats.loc['max'] != 0].sort_values())))
    
with open('method_metrics_non_zero.txt', 'w') as f:
    f.write('\n'.join(list(method_stats.columns[method_stats.loc['max'] != 0].sort_values())))

In [213]:
method = "PSO"
error = "Sammon"
metric_type = "method"

if metric_type == "class":
    all_metrics = np.array(class_metrics[2:])
    metric_list_non_zero = class_stats.columns[class_stats.loc['max'] != 0].sort_values()
elif metric_type == "method":
    all_metrics = np.array(method_metrics)
    metric_list_non_zero = method_stats.columns[method_stats.loc['max'] != 0].sort_values()
else:
    raise Exception()
    
repo_list = os.listdir(f'Results/{method}_{error}/{metric_type}')
repo_list = [x for x in repo_list if x[-5:] == '.json']
repo_list_cleaned = [x[:-5].replace('_', '/') for x in repo_list]

repos_len = len(repo_list)
metrics_len = len(metric_list_non_zero)

out = np.zeros(shape=(len(repo_list_cleaned), metrics_len-2), dtype=np.int64)

for file_idx, file_name in enumerate(repo_list):
    with open(f'Results/{method}_{error}/{metric_type}/{file_name}', 'r') as f:
        file_data = f.readline()
        file_data = json.loads(file_data)
        
    for idx, n_metrics in enumerate(range(2, len(all_metrics))):
        
        best_n_metrics = file_data[str(n_metrics)]['feat_idx']
        #print(all_metrics)
        best_n_metrics = all_metrics[best_n_metrics]
        
        bitmask = 0
        non_zero_metrics_len = 0
        for metric in best_n_metrics:
            if metric in metric_list_non_zero:
                non_zero_metrics_len += 1
                bitmask = bitmask | 2**np.where(metric_list_non_zero == metric)[0]
        
        if non_zero_metrics_len >= 2 and non_zero_metrics_len < metrics_len:
            out[file_idx, non_zero_metrics_len-2] = bitmask

            


df_out = pd.DataFrame(out, columns=list(range(2, metrics_len)))
df_out.insert(loc=0, column='Repository', value=repo_list_cleaned)

df_out.to_csv(f"{method}_{error}_{metric_type}.csv", index=False)
print(f"Number of NaN's: {(df_out == 0).sum().sum()}")

Number of NaN's: 48


In [214]:
431 + 376 + 136 + 48

991

In [203]:
df_out

,Repository,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,aio-libs/aiohttp,4112,528400,528912,791056,793104,794128,826896,1842708,1875476,1875664,1875605,2072148,2072149,2072277,2014935,2080471,2080479,2097111,2097119
1,aleju/imgaug,524304,524816,528912,791056,531984,924176,925200,794133,925205,1973782,1973783,1973911,2039383,2072151,2072471,2088663,2088671,2088927,2097119
2,ansible/ansible,4112,528400,528912,530960,793104,794128,1841684,1842708,1973780,826965,2006549,2006551,2006741,2072151,2072279,2080471,2080727,2097111,2097119
3,apache/tvm,4112,528400,528912,530960,793104,924176,925200,925201,925206,957973,1973783,1973911,2006615,2072151,2072407,2072535,2080727,2097111,2097119
4,AUTOMATIC1111/stable-diffusion-webui,4112,528400,528912,530960,793104,1580560,1842704,1842708,826901,1908245,1875541,2006613,1941079,0,2080343,2080471,2080727,0,2097119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,unifyai/ivy,4112,528400,530448,661520,661522,661523,924177,924179,1972755,1973779,990743,1023511,2039383,2072215,2080343,2072535,2096855,2097111,2097119
89,XX-net/XX-Net,524304,528400,530448,530960,793104,794128,825873,925204,925205,1973781,2006549,2006551,2006615,2072151,2072407,2072535,2080727,2097111,2097119
90,yt-dlp/yt-dlp,4112,528400,530448,530960,531984,794128,1842704,826900,1875476,1875540,1883670,1941077,2072214,2072278,2072533,2080471,2096855,2097111,2097119
91,ytdl-org/youtube-dl,4112,528400,268304,530960,531984,794128,794132,1875472,1875476,2006548,1875668,2006740,2072149,2072277,2088661,2072566,2072543,2097111,2097119


In [206]:
bin(4112)

'0b1000000010000'